In [7]:
import torch
import pandas as pd
from tqdm.notebook import trange, tqdm
import logging
import warnings
import glob
import os

In [2]:
def make_timestamp(image_path:str, camera_name: str):
    # Get timestamp from img name. TODO: move out of the loop. All these detections come from the same image, so timestamp is the same
    # splits path into folders and file and keep the last one
    # E.g.: ('../images/ie/idi/norwai/svv/ski-trd/jervskogen_1_2021-12-11_09-00-03.png') -> jervskogen_1_2021-12-11_09-00-03.png'
    _, stamp = os.path.split(image_path)

    # Remove camera name
    # E.g. 'jervskogen_1_2021-12-11_09-00-03.png' -> '2021-12-11_09-00-03.png'
    stamp = stamp.replace(camera_name + '_', '')

    # Remove file extension
    # E.g. '2021-12-11_09-00-03.png' -> '2021-12-11_09-00-03'
    stamp = stamp.replace('.png', '')

    # Split between date and time
    # E.g. '2021-12-11_09-00-03' -> '2021-12-11' + '09-00-03'
    date, time = stamp.split('_')

    # Replace hiffen in time with colon
    # E.g. '09-00-03' -> '09:00:03'
    time = time.replace('-', ':')

    # Finally, build the final timestamp string
    # E.g. '2021-12-11' + 'T' + '09:00:03' -> '2021-12-11T09:00:03'
    stamp = date + 'T' + time
    
    return stamp

In [9]:
def main():
    
    #cams = ('jervskogen_1', 'jervskogen_2', 'nilsbyen_2', 'nilsbyen_3', 'skistua', 'ronningen_1')
    cams = ( 'skistua', 'ronningen_1')
    #cam = 'nilsbyen_2'
    
    # Execute following line on cmd (cmd location -> inside project folder) of mac to mount the shared network folder on your machine.
    # User is your university's account username
    ## mount_smbfs //user@forskning.it.ntnu.no/nfs/ie/idi/norwai/svv/ski-trd ./Volumes/Ski
    ## After use unmount ->  umount /Volumes/Ski
    share_folder_path = '../sensor/Volumes/Ski/'
    
    # Few images are missing some bytes, therefore, we set turncation true 
    # which fills the missing bytes of image with grey color at the time of image loading.
    # ImageFile.LOAD_TRUNCATED_IMAGES = True 
    
    # Parameters
    MONTH_TEST = '_'            # For all images it could be empty '_' or for month '_2022-01' 
    EXP_CONF = 0.5              # Experiment Confidence

    df = dict()

    # tqdm shows the progress bar
    for cam in tqdm(cams):
            
        
        # Finds all filenames starting with the camera name
        images_path = glob.glob(share_folder_path + cam + MONTH_TEST + '*')
        
        # First, store all detections in a list, and then create a dataframe out of it
        all_timestamps_count_results = []
        only_detections_detailed_results = []
                
        # Suppress PyTorch warnings
        warnings.filterwarnings('ignore', message='User provided device_type of \'cuda\', but CUDA is not available. Disabling')
        
        # Get model
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, device='cpu')  # or yolov5m, yolov5l, yolov5x, custom

        
        for index, image_path in enumerate(tqdm(images_path)):
            try:
                
                # (A) Perform inference on machine                    
                results = model(image_path)    ## Insert 'overlapped_image'
                detected_objects = results.pandas().xyxyn[0]

                if len(detected_objects) > 0:

                    stamp = make_timestamp(image_path, cam)
                    person_count = 0

                    for index, row in detected_objects.iterrows():

                        entry_detections = {
                            "timestamp": pd.Timestamp(stamp),
                            "p1x": row['xmin'],
                            "p1y": row['ymin'],
                            "p2x": row['xmax'],
                            "p2y": row['ymax'],
                            "conf": row['confidence'],
                            "class": row['name'],
                        }

                        only_detections_detailed_results.append(entry_detections)

                        if( (row['name'] == 'person') & (row['confidence'] > EXP_CONF)):
                            person_count = person_count + 1

                    entry_timestamps = {
                        "timestamp": pd.Timestamp(stamp),
                        "count":   person_count,
                        "exp-conf": EXP_CONF,
                    }

                    all_timestamps_count_results.append(entry_timestamps)

                else:

                    stamp = make_timestamp(image_path, cam)
                    person_count = 0

                    entry_timestamps = {
                        "timestamp": pd.Timestamp(stamp),
                        "count":   person_count,
                         "exp-conf": EXP_CONF,
                    }

                    all_timestamps_count_results.append(entry_timestamps)
          

            # Error handling: if there is any error, just ignore and continue processing next images          
            except Exception as error:
                print ('Some eroor occured..continue..')
                continue

        df_only_detections_detailed_results = pd.DataFrame(only_detections_detailed_results)
        df_all_timestamps_count_results = pd.DataFrame(all_timestamps_count_results)

        
        path_string_detections = 'all/' + cam + '_only_detections_' 
        path_string_timestamps = 'all/' + cam + '_all_timestamps_count_' 
        
            
        df_only_detections_detailed_results.to_csv('../data/datasets/local/' + path_string_detections + '.csv', index=False)
        df_all_timestamps_count_results.to_csv('../data/datasets/local/' + path_string_timestamps + '.csv', index=False)
        
if __name__ == '__main__':
    main()        

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /Users/areeb/.cache/torch/hub/master.zip
fatal: not a git repository (or any of the parent directories): .git
YOLOv5 🚀 2022-3-27 torch 1.10.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 


  0%|          | 0/13598 [00:00<?, ?it/s]

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /Users/areeb/.cache/torch/hub/master.zip
fatal: not a git repository (or any of the parent directories): .git
YOLOv5 🚀 2022-3-27 torch 1.10.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 


  0%|          | 0/4107 [00:00<?, ?it/s]